In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [2]:
from datasets import load_dataset

## Load Dataset
load dataset from huggingface (can create datasets later)

In [3]:
dataset = load_dataset('aalksii/ml-arxiv-papers')

Found cached dataset parquet (/home/jerry/.cache/huggingface/datasets/aalksii___parquet/aalksii--ml-arxiv-papers-be6128a86b590f96/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'abstract'],
        num_rows: 105832
    })
    test: Dataset({
        features: ['title', 'abstract'],
        num_rows: 11760
    })
})

## Load Tokenizer for model
remember to find the tokenizer used for training original model

In [5]:
from transformers import AutoTokenizer

In [6]:
token = 'google/flan-t5-large'
# token = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(token)
x = tokenizer(text, truncation=True, max_length=2)

In [7]:
# def tokenize_function(examples):
#     return tokenizer(examples["abstract"], padding="max_length", truncation=True)

In [8]:
from datasets import concatenate_datasets

In [9]:
tokenized_title = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["title"], truncation=True), batched=True, remove_columns=["title", "abstract"])
max_source_length = max([len(x) for x in tokenized_title["input_ids"]])
print(f"Max source length: {max_source_length}")

Loading cached processed dataset at /home/jerry/.cache/huggingface/datasets/aalksii___parquet/aalksii--ml-arxiv-papers-be6128a86b590f96/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-0b9511270f4eb92b.arrow


Max source length: 70


In [10]:
tokenized_abstract = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["abstract"], truncation=True), batched=True, remove_columns=["title", "abstract"])
max_target_length = max([len(x) for x in tokenized_abstract["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/117592 [00:00<?, ? examples/s]

Max target length: 512


In [29]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["Title: " + item for item in sample["abstract"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    max_len = 2048
    # max_len = 1024
    labels = tokenizer(text_target=sample["abstract"], max_length=max_len, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["title", "abstract"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/105832 [00:00<?, ? examples/s]

Map:   0%|          | 0/11760 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [41]:
tokenized_dataset['train'][0]

{'input_ids': [11029,
  10,
  101,
  169,
  8,
  96,
  11576,
  13,
  9768,
  121,
  1295,
  13,
  180,
  1000,
  89,
  9,
  3,
  15,
  17,
  491,
  5,
  41,
  5498,
  23010,
  6503,
  61,
  12,
  8341,
  633,
  168,
  18,
  5661,
  2902,
  3438,
  7,
  5,
  242,
  284,
  13,
  135,
  6,
  62,
  428,
  46,
  17623,
  5403,
  42,
  46,
  2918,
  12628,
  21,
  10937,
  165,
  7321,
  16826,
  6,
  84,
  4105,
  7,
  8,
  15834,
  24,
  3,
  9,
  787,
  4775,
  3475,
  16,
  3,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [101,
  169,
  8,
  96,
  11576,
  13,
  9768,
  121,
  1295,
  13,
  180,
  1000,
  89,
  9,
  3,
  15,
  17,
  491,
  5,
  41,
  54

In [12]:
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

## Load model for fine tuning

In [13]:
import torch
from transformers import AutoModelForSeq2SeqLM

In [14]:
torch.cuda.device_count()

1

In [15]:
# torch.cuda.set_device('cuda:1')

In [16]:
if torch.cuda.is_available():
    print("using cuda as device")
    device = "cuda"
else:
    print("using cpu as device")
    device = "cpu"

# model_id = 'google/flan-t5-base'
model_id = 'google/flan-t5-large'
# model_id = 'declare-lab/flan-alpaca-base'
model = AutoModelForSeq2SeqLM.from_pretrained(
        model_id
)

using cuda as device


## Start Training

In [17]:
torch.cuda.device_count()

1

In [18]:
torch.cuda.current_device()

0

In [19]:
# import numpy as np
# import evaluate

# metric = evaluate.load("accuracy")

In [20]:
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [21]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [22]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_train_dataset,
#     eval_dataset=small_eval_dataset,
#     # compute_metrics=compute_metrics,
# )

In [23]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


[nltk_data] Downloading package punkt to /home/jerry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
# test to see if this can free memory
# del tokenizer

In [25]:
from transformers import pipeline

In [39]:
old_model = pipeline('text2text-generation', model=model, tokenizer=token, device=0)
result = old_model("Q: Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.")
print(result)

[{'generated_text': 'Geoffrey Hinton was a poet and writer who was born in 1791. George'}]


In [27]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = "finetune_large"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=3,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="overall_f1",
    # push to hub parameters
    push_to_hub=False,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    # compute_metrics=compute_metrics,
)


In [28]:
trainer.train()

/home/jerry/miniconda3/envs/agi/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 111012912jerry. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.894200,1.803586
2,1.831500,1.763679
3,1.776700,1.752731


TrainOutput(global_step=158748, training_loss=1.8590987087254274, metrics={'train_runtime': 55830.7994, 'train_samples_per_second': 5.687, 'train_steps_per_second': 2.843, 'total_flos': 1.0004457494642688e+17, 'train_loss': 1.8590987087254274, 'epoch': 3.0})

In [30]:
trainer.save_model()

In [31]:
new_model = pipeline('text2text-generation', model='./finetune_large', tokenizer='google/flan-t5-large', device=0)

In [37]:
result = old_model("Please write an abstract for the paper: Expected Frequency Matrices of Elections: Computation, Geometry, and Preference Learning")
print(result)

[{'generated_text': 'Expected Frequency Matrices of Elections: Computation, Geometry,'}]


In [38]:
result = new_model('Please write an abstract for the paper: Expected Frequency Matrices of Elections: Computation, Geometry, and Preference Learning')
print(result)

[{'generated_text': 'Expected Frequency Matrices of Elections: Computation, Geometry,'}]


In [42]:
dataset_id = "banking77" 

# Load raw dataset
raw_dataset = load_dataset(dataset_id)

/home/jerry/.cache/huggingface/modules/datasets_modules/datasets/banking77/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4/banking77.py:59: FutureWarning: Dataset 'banking77' is deprecated and will be deleted. Use 'PolyAI/banking77' instead.
  warnings.warn(


Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

Dataset banking77 downloaded and prepared to /home/jerry/.cache/huggingface/datasets/banking77/default/1.1.0/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [43]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3080
    })
})